# Query analysis
查询分析的意思是，将原始的查询通过llm来生成一些新的查询，将这些结果，套入retrieve中做检索。

## 查询分析中增加一些示例

随着查询变得复杂，模型在某些情况下，模型不太明确应该怎么回答问题，可以在提示中加一些例子引导llm
### 先来个查询分析

In [27]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field

sub_queries_description = """
如果原始的问题包含了多个不同的子问题，或者可以变为更加通用的问题，回答这些问题有助于回答原始问题的话，
需要写下这些相关子问题的列表
要求：
  1. 你要确保这些列表是全面的，涵盖了原始问题的所有部分
  2. 允许子问题存在冗余，确保子问题要聚焦
"""


# 定义一个model。
class Search(BaseModel):
    """搜索电影数据库"""

    query: str = Field(
        ...,
        description="应用于电影的主要相似性搜索查询",
    )
    sub_queries: List[str] = Field(
        default_factory=list, description=sub_queries_description
    )
    publish_year: Optional[int] = Field(None, description="电影发布年份")

In [28]:
## 定义查询的Chain

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """您擅长将用户问题转换为数据库查询。\
您可以访问一个关于构建基于LLM的应用程序的电影视频数据库。\
给定一个问题，返回一组经过优化以检索最相关结果的数据库查询列表。"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [29]:
from langchain.globals import set_verbose,set_debug
set_verbose(True)
set_debug(True)
query_analyzer.invoke(
    "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question>] Entering Chain run with input:
{
  "input": "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question>] s] Exiting Chain run with output:
{
  "question": "Web Voyager和Reflection Agents之间有什么区别？它们都使用LangGraph吗？"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with i

Search(query='区别: Web Voyager和Reflection Agents', sub_queries=['Web Voyager 使用 LangGraph 吗', 'Reflection Agents 使用 LangGraph 吗'], publish_year=None)

可以看到。从用户的输入中已经提取出了问题。并且拓展出了一些子问题，但是这些问题覆盖的面不够，增加example来帮助生成更好的问题。
### 在prompt中增加一些示例

为了提高性能，增加一些example

In [30]:
examples = []
question = "langchain是什么，它是一个langchain模板吗?"
query = Search(
    query="langchain是什么，它是一个langchain模板吗?",
    sub_queries=["langchain是什么", "langchain template是什么"],
)
examples.append({"input": question, "tool_calls": [query]})

In [31]:
question = "如何构建多智能体系统并从中流式传输中间步骤"
query = Search(
    query="如何构建多智能体系统并从中流式传输中间步骤",
    sub_queries=[
        "如何构建多智能体系统",
        "如何从多智能体系统中流式传输中间步骤",
        "如何流式传输中间步骤",
    ],
)

examples.append({"input": question, "tool_calls": [query]})

In [32]:
question = "LangChain agents vs LangGraph?"
query = Search(
    query="LangChain的agent和LangGraph有什么不同，如何部署他们呢",
    sub_queries=[
        "LangChain的agent是什么？",
        "LangGraph是什么？",
        "如何部署LangChain的agent？",
        "如何部署LangGraph？",
    ],
)
examples.append({"input": question, "tool_calls": [query]})

上面已经构建好了示例，下面需要将他组装在prompt中

In [33]:
import uuid
from typing import Dict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [34]:
tool_example_to_messages(examples[0])

[HumanMessage(content='langchain是什么，它是一个langchain模板吗?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '88e8e325-a1a4-4a07-a17f-f2b2200de1d8', 'type': 'function', 'function': {'name': 'Search', 'arguments': '{"query": "langchain\\u662f\\u4ec0\\u4e48\\uff0c\\u5b83\\u662f\\u4e00\\u4e2alangchain\\u6a21\\u677f\\u5417?", "sub_queries": ["langchain\\u662f\\u4ec0\\u4e48", "langchain template\\u662f\\u4ec0\\u4e48"], "publish_year": null}'}}]}, tool_calls=[{'name': 'Search', 'args': {'query': 'langchain是什么，它是一个langchain模板吗?', 'sub_queries': ['langchain是什么', 'langchain template是什么'], 'publish_year': None}, 'id': '88e8e325-a1a4-4a07-a17f-f2b2200de1d8'}]),
 ToolMessage(content='You have correctly called this tool.', tool_call_id='88e8e325-a1a4-4a07-a17f-f2b2200de1d8')]

带入一个example看，一个example生成了三个message，这代表一次完整的响应，首先，HumanMessage代表人问AI的问题，AIMessage代表AI需要调用 Search工具，ToolMessage 代表工具的输出。

In [35]:
# 现在生成了示例的prompt
example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]

In [36]:
# 下面只需要将所有的链接在一起就好了
query_analyzer_with_examples = (
    {"question": RunnablePassthrough()}
    | prompt.partial(examples=example_msgs)
    | structured_llm
)

In [37]:
from langchain.globals import set_debug
set_debug(True)
query_analyzer_with_examples.invoke(
    "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question>] Entering Chain run with input:
{
  "input": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question>] s] Exiting Chain run with output:
{
  "question": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "刘德华和吴彦祖，到底谁帅呢？谁是彦祖呢？"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/sta

Search(query='刘德华和吴彦祖谁更帅，吴彦祖是谁', sub_queries=['刘德华帅吗？', '吴彦祖帅吗？', '吴彦祖是谁？'], publish_year=None)

## 没有生成查询分析的情况改怎么处理

这种情况下从本质上区分不了，根本原因是压根儿没有子问题，只能从代码上做考虑，解析Response如果有子问题就返回子问题，没有就返回原始的响应。在没有什么别的好的处理

In [45]:
from langchain_core.runnables import chain
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

output_parser = PydanticToolsParser(tools=[Search])

structured_llm = llm.bind_tools([Search])
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm
@chain
def custom_chain(question):
    response = query_analyzer.invoke(question)
    if "tool_calls" in response.additional_kwargs:
        query = output_parser.invoke(response)
        # 这里可以做业务逻辑，比如在套入一个chain或者。。。
        # Could add more logic - like another LLM call - here
        print("*" * 100)
        return query
    else:
        return response

In [50]:
custom_chain.invoke("where did Harrison Work")

[chain/start] [chain:custom_chain] Entering Chain run with input:
{
  "input": "where did Harrison Work"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "where did Harrison Work"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question>] Entering Chain run with input:
{
  "input": "where did Harrison Work"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "where did Harrison Work"
}
[chain/end] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "where did Harrison Work"
}
[chain/end] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question>] s] Exiting Chain run with output:
{
  "question": "where did Harrison Work"
}
[chain/start] [chain:custom_cha

[Search(query='Harrison Ford', sub_queries=['Where did Harrison Ford work?'], publish_year=None)]

In [51]:
custom_chain.invoke("hi!")

[chain/start] [chain:custom_chain] Entering Chain run with input:
{
  "input": "hi!"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "hi!"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question>] Entering Chain run with input:
{
  "input": "hi!"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "hi!"
}
[chain/end] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "hi!"
}
[chain/end] [chain:custom_chain > chain:RunnableSequence > chain:RunnableParallel<question>] s] Exiting Chain run with output:
{
  "question": "hi!"
}
[chain/start] [chain:custom_chain > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "hi!"
}
[chain/

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 266, 'total_tokens': 276}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2edeb9e6-d60e-44e2-bfbb-9aa068967732-0', usage_metadata={'input_tokens': 266, 'output_tokens': 10, 'total_tokens': 276})

这里的思路是自己解析Response，如果有工具调用就走工具调用，然后就可以自己做逻辑，否则就直接返回

## 处理多个查询

有时，查询分析技术可能允许生成多个查询。在这些情况下，需要记住运行所有的查询，然后将结果合并。将展示一个简单的例子（使用模拟数据）来说明如何做到这一点。

这里就是使用async，将上面的chain结合起来，自定义个chain

In [58]:
set_debug(False)
@chain
async def custom_chain(question):
    response = await query_analyzer.ainvoke(question)
    docs = [response]
    # for query in response.queries:
    #     # 这里做一些额外的处理
    #     # new_docs = await retriever.ainvoke(query)
    #     docs.extend("new_docs")
    # # You probably want to think about reranking or deduplicating documents here
    # # But that is a separate topic
    return docs

In [59]:
await custom_chain.ainvoke("where did Harrison Work")

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BhRUTWdBhhINAGuah3faaxqN', 'function': {'arguments': '{"query":"Harrison Work"}', 'name': 'Search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 268, 'total_tokens': 283}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a4a3e7e-f347-451b-84c5-1164c402b5e4-0', tool_calls=[{'name': 'Search', 'args': {'query': 'Harrison Work'}, 'id': 'call_BhRUTWdBhhINAGuah3faaxqN'}], usage_metadata={'input_tokens': 268, 'output_tokens': 15, 'total_tokens': 283})]

In [60]:
await custom_chain.ainvoke("where did Harrison and ankush Work")

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_thnLQggH7i65wH5YtiRaCbgJ', 'function': {'arguments': '{"query": "Workplace of Harrison", "sub_queries": ["Harrison\'s workplace"]}', 'name': 'Search', 'parameters': None}, 'type': 'function'}, {'id': 'call_lFar2Cjq9XYFVom2uc5Y4zDe', 'function': {'arguments': '{"query": "Workplace of Ankush", "sub_queries": ["Ankush\'s workplace"]}', 'name': 'Search', 'parameters': None}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 271, 'total_tokens': 337}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-483f12b9-0e95-4898-9243-647f39105c7e-0', tool_calls=[{'name': 'Search', 'args': {'query': 'Workplace of Harrison', 'sub_queries': ["Harrison's workplace"]}, 'id': 'call_thnLQggH7i65wH5YtiRaCbgJ'}, {'name': 'Search', 'args': {'query': 'Workplace of Ankush', 'sub_queries': ["Ankush's workplace"]}, 'id': 'cal

## 处理多个检索
原理：他就是让生成的查询中返回了对应的key，从而找到对应的检索器，这样就可以做具体的检索了。

In [62]:
# 在chroma中添加一些数据

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

texts = ["Harrison worked at Kensho"]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(texts, embeddings, collection_name="harrison")
retriever_harrison = vectorstore.as_retriever(search_kwargs={"k": 1})

texts = ["Ankush worked at Facebook"]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(texts, embeddings, collection_name="ankush")
retriever_ankush = vectorstore.as_retriever(search_kwargs={"k": 1})

In [65]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search for information about a person."""

    query: str = Field(
        ...,
        description="Query to look up",
    )
    person: str = Field(
        ..., # 指定资料类型
        description="Person to look things up for. Should be `HARRISON` or `ANKUSH`.",
    )

In [66]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

output_parser = PydanticToolsParser(tools=[Search])

system = """You have the ability to issue search queries to get information to help answer user information."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [67]:
query_analyzer.invoke("where did Harrison Work")

Search(query='work history', person='HARRISON')

In [68]:
query_analyzer.invoke("where did ankush Work")

Search(query='work', person='ANKUSH')

可以将person中的值和检索器结合起来

In [69]:
retrievers = {
    "HARRISON": retriever_harrison,
    "ANKUSH": retriever_ankush,
}
@chain
def custom_chain(question):
    response = query_analyzer.invoke(question)
    retriever = retrievers[response.person]
    # 这里最终调用检索器来做操作
    return retriever.invoke(response.query)
# 这里
custom_chain.invoke("where did Harrison Work")

[Document(page_content='Harrison worked at Kensho')]

In [70]:
custom_chain.invoke("where did ankush Work")

[Document(page_content='Ankush worked at Facebook')]

## 构造filters

langchan提供了方法，可以将查询拓展出来的结构转换为对应检索器的查询语句，比如es的查询语句，chroma的查询语句


In [71]:
from typing import Optional

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain_community.query_constructors.chroma import ChromaTranslator
from langchain_community.query_constructors.elasticsearch import ElasticsearchTranslator
from langchain_core.pydantic_v1 import BaseModel
class Search(BaseModel):
    query: str
    start_year: Optional[int]
    author: Optional[str]

In [72]:
# 这里的query就可以通过前面的查询分析来做
search_query = Search(query="RAG", start_year=2022, author="LangChain")

In [76]:
def construct_comparisons(query: Search):
    comparisons = []
    if query.start_year is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GT,
                attribute="start_year",
                value=query.start_year,
            )
        )
    if query.author is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="author",
                value=query.author,
            )
        )
    return comparisons

In [77]:
comparisons = construct_comparisons(search_query)

In [78]:
_filter = Operation(operator=Operator.AND, arguments=comparisons)

到这里，都是langchain独立，自己提供的查询的filter语句

In [79]:
# 转换为es
ElasticsearchTranslator().visit_operation(_filter)

{'bool': {'must': [{'range': {'metadata.start_year': {'gt': 2022}}},
   {'term': {'metadata.author.keyword': 'LangChain'}}]}}

In [80]:
# 转换为 Chroma的语句
ChromaTranslator().visit_operation(_filter)

{'$and': [{'start_year': {'$gt': 2022}}, {'author': {'$eq': 'LangChain'}}]}

## 和Retrieve的结合

生成查询分析的时候可以和Retrieve结合，让模型生成的问题在既定范围之内，而不是随意发散，简单的处理方式将所有的值全部塞给模型，这样会导致，context过长，信息密度低，可以和Retrieve结合
减少context的长度。

In [3]:
from faker import Faker
# 制造一些假数据
fake = Faker()
names = [fake.name() for _ in range(1000)]
print(names[0])
print(names[10])
print(names[20])

Andrea Rodriguez
Jacob Diaz
Darius Gates


In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

# 定义结构化输出
class Search(BaseModel):
    query: str
    author: str
    
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 写基础的base prompt
system = """Generate a relevant search query for a library system"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [6]:
query_analyzer.invoke("what are books about aliens by Andrea Rodriguez")

Search(query='books about aliens', author='Andrea Rodriguez')

注意看这里，有时候写的时候时候可能会写错，这里将错误的问题传递给llm，llm也会生成错误的答案，

In [8]:
query_analyzer.invoke("what are books about aliens by adrea Rodriguez")

Search(query='books about aliens', author='Adrea Rodriguez')

简单的处理方式，将所有的数据在给大模型。这是最简单的方式，但是会导致context过长

In [10]:

system = """Generate a relevant search query for a library system.

`author` attribute MUST be one of:

{authors}

Do NOT hallucinate author name!"""
base_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
prompt = base_prompt.partial(authors=", ".join(names))

In [11]:
query_analyzer_all = {"question": RunnablePassthrough()} | prompt | structured_llm

In [13]:
from langchain.globals import set_debug
set_debug(True)
query_analyzer_all.invoke("what are books about aliens by adrea Rodriguez")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question>] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "what are books about aliens by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question>] s] Exiting Chain run with output:
{
  "question": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "what are boo

Search(query='books about aliens', author='Andrea Rodriguez')

将所有的数据，传递给大模型，这样再问他错误的问题的时候，他会自动修复 。
还有一种方式是通过向量数据库搜索。在传递给模型之前先做一遍，先走一遍向量化搜索，这样可以保证有限的上下文 。

In [15]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 构建向量数据库
vectorstore = Chroma.from_texts(names, embeddings, collection_name="author_names")

In [16]:
# 从向量数据库中先做一遍筛选
def select_names(question):
    _docs = vectorstore.similarity_search(question, k=10)
    _names = [d.page_content for d in _docs]
    return ", ".join(_names)

In [17]:
create_prompt = {
    "question": RunnablePassthrough(),
    "authors": select_names,
} | base_prompt

In [18]:
query_analyzer_select = create_prompt | structured_llm

In [19]:
create_prompt.invoke("what are books by adrea Rodriguez")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what are books by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors>] Entering Chain run with input:
{
  "input": "what are books by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "what are books by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "what are books by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:select_names] Entering Chain run with input:
{
  "input": "what are books by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:select_names] [1.14s] Exiting Chain run with output:
{
  

ChatPromptValue(messages=[SystemMessage(content='Generate a relevant search query for a library system.\n\n`author` attribute MUST be one of:\n\nAndrea Rodriguez, Andrea Rodriguez, Jonathan Rodriguez, Jonathan Rodriguez, Patricia Rodriguez, Patricia Rodriguez, Michael Rodriguez, Michael Rodriguez, Andrea Jones, Andrea Jones\n\nDo NOT hallucinate author name!'), HumanMessage(content='what are books by adrea Rodriguez')])

这里可以看到传递给模型的prompt中已经有相关的答案了。

In [20]:
query_analyzer_select.invoke("what are books about aliens by adrea Rodriguez")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors>] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "what are books about aliens by adrea Rodriguez"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,authors> > chain:select_names] Entering Chain run with input:
{
  "input": "what are books about aliens by adrea Rodriguez"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,authors> 

Search(query='aliens', author='Andrea Rodriguez')